In [8]:
import pandas as pd
import numpy as np
from collections import defaultdict
import scipy
import scipy.stats
from statsmodels.stats.multitest import multipletests


def filter_functions(callstacks, names):
  result = [[function for function in callstack if function in names] for callstack in callstacks]
  return [callstack for callstack in result if len(callstack) > 1]

class Capture:
  # is used to throw out the initialization period when the benchmark has not been started yet, but the 
  # timetime is excessively long
  def __init__(self, callstack_csv_path, frame_mean, frame_ends, init_end_time): 
    self.frame_mean = frame_mean
    self.frame_ends = frame_ends
    self.callstack_csv = pd.read_csv(callstack_csv_path).to_numpy()
    
    self.callstack_csv = self.callstack_csv[self.callstack_csv[:,1] > init_end_time,:]
    
    self.callstack_csv = self.callstack_csv[self.callstack_csv[:, 1].argsort()] # sort by timestamp
    self.frame_index_to_callstacks = self.get_frame_index_to_callstacks()
    self.total_callstacks = sum(len(s) for s in self.frame_index_to_callstacks.values())

  def filter_functions(self, names):
    filterred = {}
    for frame_index, callstacks in self.frame_index_to_callstacks.items():
      filterred[frame_index] = filter_functions(callstacks, names)
    self.frame_index_to_callstacks = filterred

  # returns a dict frame_index -> List[Callstack]
  def get_frame_index_to_callstacks(self): 
    frame_index_to_callstacks = {}
    frame_index = 0
    frame_end = self.frame_ends[frame_index]
    frame_index_to_callstacks[frame_index] = []
    callstack_names = [s.split('/') for s in  self.callstack_csv[:, 2]]
    timestamps = self.callstack_csv[:, 1]
    for i in range(self.callstack_csv.shape[0]):
      timestamp = timestamps[i]

      while timestamp > frame_end:
        if frame_index + 1 == self.frame_ends.shape[0]: 
          return frame_index_to_callstacks
        frame_index += 1
        frame_end = self.frame_ends[frame_index]
        frame_index_to_callstacks[frame_index] = [] 
      frame_index_to_callstacks[frame_index].append(callstack_names[i])
    return frame_index_to_callstacks

  def get_exclusive_count(self, function_name):
    exclusive_count = 0;
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        if (len(callstack) > 0 and callstack[0] == function_name):
          exclusive_count += 1
    return exclusive_count

  def avg_frame_time(self):
    return self.frame_mean / 1e6

  def total_callstacks(self):
    return self.total_callstacks

  def get_exclusive_rate(self):
    name_to_hits = {}
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        head = callstack[0]
        name_to_hits.setdefault(head, 0)
        name_to_hits[head] += 1
    total_callstacks = self.total_callstacks
    return defaultdict(lambda : 0, dict([(name, (hits / total_callstacks)) for name, hits in name_to_hits.items()]))

  def time_per_frame(self): 
    mean_frame_time = self.avg_frame_time()
    return defaultdict(lambda : 0, dict([(name, (mean_frame_time  * rate)) for name, rate in self.get_exclusive_rate().items()]))

  def function_names(self):
    result = []
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        result += callstack
    return list(set(result))


In [19]:
def load_capture_with_frametrack(callstack_csv_path, frametrack_csv_path, max_frametime):
  frametrack_csv = pd.read_csv(frametrack_csv_path).to_numpy()
  frametrack_csv = frametrack_csv[frametrack_csv[:, 3].argsort()] # sort by end
  frame_durations = frametrack_csv[:, 4]
  frame_mean = frame_durations.mean()
  frame_ends = frametrack_csv[:, 3]
  last_init_frame = np.max((frame_durations > max_frametime).nonzero())
  init_end_time = frametrack_csv[last_init_frame, 3]

  frame_ends = frame_ends[last_init_frame:]
  frame_mean = frame_durations[last_init_frame:].mean()
  
  return Capture(callstack_csv_path, frame_mean, frame_ends, init_end_time)

In [12]:
import json 

def load_cumbia_capture_with_results_json(callstack_csv_path, results_json_path, max_frametime):
  results = json.load(open(results_json_path))
  fps = results["outputs"][1]['values']
  fps = np.array(fps[1:])
  times = results["outputs"][1]['times'] 
  frame_mean = np.dot(np.diff(times), 1 / fps)/np.diff(times).sum()
  start_time_ns = result1["startTime"]['msSinceEpoch'] * 1e6
  

In [13]:
load_cumbia_capture_with_results_json("~/cumbia_callstack4_thread.csv", '/usr/local/google/home/avanesov/results_fhd.json', 85e6)

(0.02162144765561394, 0.02141929109714461, 0.021029330655529428)

In [170]:
capture2 = load_capture_with_frametrack("~/cumbia_callstack4_thread.csv", "~/cumbia_frametrack4.csv",  85*1e6)
capture1 = load_capture_with_frametrack("~/cumbia_callstack5_thread.csv", "~/cumbia_frametrack5.csv",  85*1e6)


names1 = capture1.function_names()
names2 = capture2.function_names()
names = set(names1).intersection(set(names2))
names.remove("???")
capture1.filter_functions(names)
capture2.filter_functions(names)

In [14]:
import json
result1 = json.load(open('/usr/local/google/home/avanesov/results_fhd.json'))


In [15]:
fps = result1["outputs"][1]['values']
fps = np.array(fps[1:])
times = result1["outputs"][1]['times'] 

In [18]:
result1["startTime"]['msSinceEpoch'] * 1e6

1.651507800708e+18

In [57]:
np.dot(np.diff(times), 1 / np.array(fps))/np.diff(times).sum(), ( 1 / np.array(fps)).mean()

(0.02162144765561394, 0.02141929109714461)

In [137]:
def overhead_per_function(capture1, capture2):
  avg_frame_time1 = capture1.avg_frame_time()
  avg_frame_time2 = capture2.avg_frame_time()

  time_per_frame1 = capture1.time_per_frame()
  time_per_frame2 = capture2.time_per_frame()

  result = []
  for name in names:
    time1 = time_per_frame1[name]
    time2 = time_per_frame2[name]
    result.append((name, (time1 - time2) / (avg_frame_time1 - avg_frame_time2)))
  return result


In [138]:
overheads = overhead_per_function(capture1, capture2)
overheads.sort(key = lambda x: -x[1])
overheads_dict = dict(overheads)
names_with_high_overhead = [name for name in names if overheads_dict[name] > 0.01]

In [171]:
# capture1 -- slow
def test(capture1, rates1, capture2, rates2, name):
  n1 = capture1.total_callstacks
  n2 = capture2.total_callstacks
  p1 = rates1[name] 
  p2 = rates2[name]  
  f1 = capture1.avg_frame_time()
  f2 = capture2.avg_frame_time()

  stat = f1*p1 - f2*p2
  stat_var = f1*p1*(f1-f1*p1)/n1 + f2*p2*(f2-f2*p2)/n2
  stat = stat / np.sqrt(stat_var)
  p = scipy.stats.norm().cdf(stat)
  if (np.isnan(p)): 
    return 1.0
  return min(p, 1-p) * 2

  

In [172]:
rate1 = capture1.get_exclusive_rate()
rate2 = capture2.get_exclusive_rate() 
pvalues = [(name, test(capture1, rate1, capture2, rate2, name)) for name in names]

/tmp/ipykernel_3154535/2019495495.py:12: RuntimeWarning: invalid value encountered in double_scalars
  stat = stat / np.sqrt(stat_var)


In [173]:
reject, corrected,_,_ = multipletests([pvalue for name, pvalue in pvalues])

/usr/local/google/home/avanesov/.local/lib/python3.9/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


In [174]:
corrected[corrected < 0.05]

array([4.13821003e-03, 4.49422535e-08])

In [175]:
namesnp = np.array([name for name, pvalue in pvalues])
rejected_names = namesnp[np.array(reject)]

In [176]:
len(rejected_names), len(names)

(2, 1528)

In [177]:
rejected_names

array(['eva::graphics::rendering::rendering_options::draw_wireframe() const',
       'eva::graphics::rendering::spatial_geometry::geometry() const'],
      dtype='<U1862')

In [ ]:
pvalues.sort(key = lambda x : x[1]) 


In [ ]:
(pvalues)

In [ ]:
pvalues

In [ ]:
problem_name= pvalues[0][0]
rate1[problem_name], rate2[problem_name]

In [ ]:
test(capture1, rate1, capture2, rate2, problem_name)

In [ ]:
len(set(rejected_names).intersection(names_with_high_overhead))

In [ ]:
names_with_high_overhead